# このプログラムは、人手でエンティティ同定作業を行うためのGUIを作成します。

作業に必要なデータは、

1. 文書と、その中の曖昧性のあるメンション及びエンティティ候補が保存されているデータ(hogehoge+mention.json)

1. エンティティ名から所在都道府県を出す辞書データ(entity_pref.json) の 2つです。

作業を進めると、1のデータは書き換えられていき、すべて同定し終わって1のデータ内から曖昧性のあるメンションが無くなったら作業終了です。

2024年2月現在、now81上では jupyter notebook が動作しないっぽいので、このプログラムと上の必要なデータはローカルにダウンロードするか、Google Colab 上で動かしてください。

まず、必要なライブラリを読み込みます。

In [45]:
import json
from ipywidgets import HTML,Button,HBox,Output
from IPython.display import display,clear_output
import time
import csv
import shutil

prefectures = [
    '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県',
    '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県',
    '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県',
    '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県',
    '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県'
]

GUIを表示するためのクラスです。

In [49]:
class Manual_entity_identification():
    def __init__(self, target_data, target_data_type='test', entity_pref_data = 'entity_pref.json'):
        
        # 保存するとき何か起こるとまずいので、最初にデータのバックアップをとる
        shutil.copy(target_data, target_data.replace('.json', '_backup.json'))

        with open(target_data,'r', encoding='utf8') as f:
            self.tweet_data = json.load(f)

        with open(entity_pref_data,'r', encoding='utf8') as f:
            self.ent2pref = json.load(f)
                    
        self.ents_tmp = []
        self.target_num = 0
        self.target = {}
        self.target_mention = ''
        self.save_name = target_data
        self.target_type = target_data_type
        
        self.problem = self.next_problem_generator()

    def show_selector(self):
        "選択肢の一覧を表示する関数"
        
        num = self.target_num
        text = self.target['text'] + '<br>' + self.target['location']
        mention = self.target_mention
        ents = self.target['candidate'][mention]
        label = self.target['label']
        
        print(f'{num+1}/{len(self.tweet_data[self.target_type])}')
        mention_emp = f'<span style="background-color: #4db6ac">{mention}</span>'
        text = text.replace(mention,mention_emp) + f'<br>正解ラベル:{label}'
        display(HTML(value=text))
        
        self.ents_tmp = []
        cnt = 0

        for k,_ in sorted(ents.items(), key=lambda x:x[1], reverse=True):
            button = Button(description=k)
            button.value = cnt
            button.on_click(self.clicked)
            try:
                pref = self.ent2pref[k]
            except KeyError:
                pref = ''
            html_text = f'''<h4><a href="https://ja.wikipedia.org/wiki/{k}"
                target="_blank" rel="noopener noreferrer">{k}</a> [{pref}]</h4>'''
            html = HTML(value=html_text)
            self.ents_tmp.append(k)
            hbox = HBox([button,html])
            display(hbox)
            cnt += 1
            
        imp_button = Button(description='該当なし')
        imp_button.on_click(self.impossible)
        display(imp_button)
            
        end_button = Button(description='終了', button_style='danger')
        end_button.on_click(self.end)
        display(end_button)
            
    def clicked(self, b):
        "「該当なし」　が押された場合の処理です"
        self.target['Entity'][self.target_mention] = self.ents_tmp[b.value]
        clear_output()
        if b.value: print(f'「{self.target_mention}」⇒「{self.ents_tmp[b.value]}」')
        next(self.problem)        
            
    def impossible(self, b):
        "「該当なし」　が押された場合の処理です"
        clear_output()
        next(self.problem)
        
    def next_problem_generator(self):
        "次のエンティティ同定対象を表示させるジェネレータ関数です。対象が見つかったらセレクタを呼び出して戻ります。"
        for i in range(self.target_num, len(self.tweet_data[self.target_type])):
            if 'Entity' in self.tweet_data[self.target_type][i]:
                continue
            self.target = self.tweet_data[self.target_type][i]
            self.target['Entity'] = {}
            self.target_num = i
            tmp = self.target['candidate'].copy()
            for men in tmp:
                # メンションに対するエンティティ候補が1つしかない場合はそれを自動で選択します。
                # 厳密にはそのエンティティ候補が間違いである可能性もあると思います。その場合は「該当なし」を押さなければいけません。
                # その設定で行いたい場合は下3行をコメントアウトしてください。
                if len(self.target['candidate'][men]) == 1:
                    self.target['Entity'][men] = next(iter(self.target['candidate'][men]))
                    continue
                    
                self.target_mention = men
                self.show_selector()
                yield
        else:
            self.end(None)
            yield
        
    def end(self, b):
        "「終了」　が押された場合、あるいは作業が全て終了したときに呼び出される"
        if b is not None:
            # 作業中に押された場合、今出ている選択肢分は破棄する
            del self.target['Entity']
            
        with open(self.save_name, 'w', encoding='utf8') as f:
            json.dump(self.tweet_data, f, indent=2, ensure_ascii=False)
        display(HTML('<h1><b>人手エンティティ同定作業終了！ お疲れ様でした！！<b><h1>'))

ここまでの読み込みが済んだら、後は以下のセルを実行して作業を行います。
やめたくなったら終了ボタンを押してください。それまでの作業結果が保存され、次回はその続きから作業を再開できます。

## 必ず終了ボタンを押して終了してください。終了ボタンを押さないと結果が保存されません。

In [51]:
mei = Manual_entity_identification('tweet_data+mention.json')
next(mei.problem)

HTML(value='<h1><b>人手エンティティ同定作業終了！ お疲れ様でした！！<b><h1>')

以下は私がこの作業を行った際、**一番上のエンティティ以外を選んだとき**のその理由をメモしていたものです。参考になるかは分かりませんが、添付しておきます。  
実は、一番上のエンティティ以外を押すと、次の選択肢が出るときに一番上に選んだメンションとエンティティの対応が「メンション」⇒「エンティティ」の形で出ます。お役立てください。

- 「八木」→「八木西口駅」ポピュラーなのは広島県の八木だが、ここでは奈良県の八木西口駅を指していた。駅名と言う情報から判断。
- 「ディズニーランド」→「東京ディズニーランド」最初に出てくるのがアメリカのディズニーランドなため、東京ディズニーランドを選択。頻出。
- 「中央区」→「中央区（相模原市）」単純に相模原市中央区と書いてあったため。
- 「新幹線」→「東海道新幹線」最初の新幹線は包含的な新幹線なため、静岡と言う所在地からこちらを選択。頻出。
- 「ヨコハマ」⇒「横浜市」第一候補が横浜ゴムだった。
- 「神奈川県横浜市」⇒「横浜市」第一候補が、市名としての神奈川だったため。
- 「長良ヶ丘〜」⇒「長良村」第一候補がテレビ中継局だった。
- 「神岡」⇒「神岡町 (岐阜県)」第一候補が中継局だった。
- 「マックスバリュ南15条店」⇒「マックスバリュ北海道」マックスバリューの会社としてバリエーションがあったため、投稿地の北海道を選択。
- 「板橋」⇒「板橋区」板橋区板橋か、板橋区そのものかで区の方を選択。
- 「NGK」⇒「なんばグランド花月」略語だが、第一候補ではなかった。やや頻出。
- 「大阪府岸和田市」⇒「岸和田市」大阪府ではなく、岸和田市を選択。
- 「石川県輪島市」⇒「輪島市」石川県ではなく、輪島市を選択。
- 「三沢」⇒「三沢市」飛行場や高校へのリンクが先頭にあった。
- 「大宮」⇒「大宮市」先頭が大宮駅だった。
- 「浦和」⇒「浦和区」先頭がスポーツチーム。
- 「小倉一門」⇒「小倉市」先頭が競馬場。
- 「大垣」⇒「大垣市」第一候補が駅名。
- 「YOKOHAMA」⇒「横浜市」先頭がゴム会社。
- 「十勝」⇒「十勝郡」先頭が役所名。
- 「富士急」⇒「富士急ハイランド」先頭が富士急行。推測。頻出。
- 「@ 大里屋本店」⇒「大里郡」同名の地名が沖縄、福岡、千葉...にある中で正解ラベルから推測。
- 「甲府」⇒「甲府市」先頭がサッカーチームだった。
- 「中央区」⇒「中央区 (さいたま市)」数ある中央区の中からさいたま市中央区を選択。頻出。
- 「千葉県千葉市若葉区」⇒「千葉市」なぜか先頭が三重県だった。
- 「三国」⇒「三国町」先頭が競艇場。
- 「大宮」⇒「大宮市」先頭が駅。頻出。
- 「阿蘇」⇒「阿蘇市」先頭が阿蘇山だったが、文脈的に阿蘇山のことではないと判断。
- 「@ 上越」⇒「上越市」先頭が上越新幹線で、文脈判断。
- 「万代口」⇒「万代 (新潟市)」先頭がチェーンストアの名前でした。
- 「宝塚」⇒「宝塚市」先頭が宝塚歌劇団だった。文脈的に判断。
- 「学園前」⇒「学園前駅 (北海道)」奈良や千葉にも同名駅があり。文脈判断。
- 「下北」⇒「下北沢」判断が難しかったが、いわゆる下北は下北半島ではなく東京都の方だと判断。やや頻出。
- 「南区」⇒「南区 (札幌市)」数ある南区の中から札幌市のものを選択。
- 「本宿駅」⇒「本宿駅 (群馬県)」同名の駅が愛知県にも存在。
- 「元町」⇒「元町 (神戸市)」同名の町が多数存在。先頭は横浜市。
- 「市電」⇒「鹿児島市交通局」かなり一般的な名詞だが、鹿児島市のものと文脈判断できた。
- 「蒲郡」⇒「蒲郡市」先頭が競艇場。
- 「野間灯台」⇒「野間町」先頭が福岡のものだった。
- 「美浜」⇒「美浜町 (愛知県)」千葉県、福井県にも同名の地名が存在。
- 「草津」⇒「草津町」先頭がサッカーチーム。やや頻出。
- 「[千葉県松戸市」⇒「松戸市」なぜか先頭が熊本。
- 「常滑」⇒「常滑市」先頭が競艇場。やや頻出。
- 「大原」⇒「大原 (竹富町)」同名の地名が複数存在。文脈で特定。
- 「小浜」⇒「小浜町 (長崎県)」同名の地名が複数存在。文脈と正解ラベルから特定。
- 「長野県上田市」⇒「上田市」先頭が「長野県」エンティティだった。メンション的には上田市を指すと判断。
- 「北海道札幌市中央区南2条西2丁目札幌NSビル２F」⇒「中央区 (札幌市)」札幌市エンティティが先に来ていたが、メンション的にはこちらと判断。
- 「Fuji Mihana」⇒「富士山」先頭が異なる企業名だった。
- 「エビス」⇒「恵比須駅」先頭がビール会社を指していた。
- 「川平」⇒「川平湾」沖縄の地名。先頭には島根の地名が来ていた。
- 「栄」⇒「栄 (我孫子市)」地元周辺なので分かったが、千葉県の地名を列挙している中にあったのでこれは我孫子。
- 「旭川」⇒「旭川市」先頭が岡山の地名となっていた。頻出。
- 「南紀勝浦」⇒「南紀勝浦温泉」先頭がその地域の地名だったが、文脈的に温泉そのものを指していたので。
- 「広島県広島市」⇒「広島市」先頭が「広島県」エンティティだった。より細かなエンティティにするパターン。
- 「石山」⇒「石山 (新潟市)」同名の地名が多いパターン。文脈判断。
- 「竹野」⇒「竹野浜」先頭が中学校の名前だった。
- 「本町」⇒「本町 (仙台市)」同名の地名が多いパターン。
- 「小倉」⇒「小倉市」先頭が競馬場。やや頻出。
- 「北野」⇒「北野 (伊丹市)」同名の地名が多いパターン。
- 「@ 上河内」⇒「上河内町」先頭が神奈川県海老名市の地名。
- 「長谷寺」⇒「長谷寺 (鎌倉市)」奈良県のものを筆頭に、同名の寺が多数存在。
- 「高雄」⇒「高雄 (京都市)」同名のものが熊本県にも地名として存在。正解ラベルより判定。
- 「宮古」⇒「宮古島」岩手県の宮古市が先に来るが、文脈的に沖縄の方でとった。
- 「ポルタ」⇒「京都駅前地下街ポルタ」同じ名称が複数存在。文脈より推定。
- 「錦町」⇒「錦町 (仙台市)」同名の地名が複数存在パターン。
- 「柏」⇒「柏市」先頭が柏レイソル。
- 「青島」⇒「青島 (愛媛県)」同名の島・地名が存在。文脈推定。
- 「京橋」⇒「京橋 (大阪市)」同名の地名シリーズ。やや頻出。
- 「国立近代美術館」⇒「東京国立近代美術館」先頭がフランスの「国立近代美術館」でした。
- 「新富士」⇒「新富士駅 (静岡県)」北海道の同名駅が第一候補。
- 「東京都足立区」⇒「足立区」なぜか先頭が埼玉県。
- 「御前山青年」⇒「御前山村」同名の地名が東京にも存在した。
- 「三浦三」⇒「三浦半島」海鮮に関する文脈だったのでここだと推定。正解ラベルは東京だが。
- 「国営みちのく」⇒「東北地方」判断に迷ったが、ここから得られる情報は東北地方であることだと考えたのでこれにした。
- 「河原町」⇒「河原町通」先頭が鳥取県の河原町だったが、これは京都旅行の文脈だったため。
- 「一乗寺」⇒「一乗寺 (京都市の地名)」先頭は兵庫県にある寺だったが、京都旅行の文脈で登場。
- 「高砂」⇒「高砂 (葛飾区)」さいたまや兵庫にも同名地名が存在。
- 「船橋」⇒「船橋市」先頭が船橋競馬場だった。
- 「市川」⇒「市川市」先頭が兵庫県にある市川だったが、船橋・市川と並んでいたため、これは千葉県の市川。
- 「九十九島」⇒「九十九島 (島原市)」秋田、愛媛に同名地名が存在。
- 「祇園北高」⇒「祇園 (広島市)」先頭がお笑いコンビだった。
- 「地下鉄東西線」⇒「仙台市地下鉄東西線」京都、東京、札幌にも「地下鉄東西線」が存在。
- 「総武快速」⇒「横須賀・総武快速線」別の総務快速線が第一候補だったため。
- 「大井」⇒「大井 (品川区)」先頭が競馬場。やや頻出。
- 「伊丹」⇒「大阪国際空港」伊丹まで飛行機に乗るという表現から伊丹市ではないと判断。
- 「山崎」⇒「山崎 (鎌倉市)」同名の地名が各地に存在。
- 「浦和」⇒「浦和区」先頭がサッカーチームの浦和レッズだったた。
- 「鈴鹿」⇒「鈴鹿」先頭が鈴鹿サーキットだったが、文脈的に違うと判断。
- 「磐田」⇒「磐田市」先頭がサッカーチーム。
- 「白浜大浜」⇒「白浜 (下田市)」同名地名が複数存在し、文脈より所在地を推定。
- 「大室山」⇒「大室山 (静岡県)」山梨と静岡にまたがるため、正解ラベル側のものを選択。
- 「鹿島観光」⇒「鹿島市」先頭が鹿島アントラーズ（サッカーチーム）
- 「城北城東」⇒「城北中学校・高等学校」地名と文脈から判断。
- 「新三郷」⇒「新三郷駅」先頭は地名を指していたが、文脈より駅名が妥当と判断。
- 「しまね旅」⇒「島根県」先頭が信用金庫だった。
- 「オホーツク」⇒「オホーツク海」先頭が総合振興局だったため。
- 「岐阜県岐阜市」⇒「岐阜市」なぜか先頭が栃木県を指していたため。
- 「北区」⇒「北区 (東京都)」津々浦々な北区の中から。大阪が１位らしい。
- 「東山」⇒「東山 (目黒区)」有名なのは京都のもの。文脈より東京と判定。
- 「小川町」⇒「小川町 (小平市)」これは東京の地名らしい。同名地名が複数各地に存在。
- 「東陽町」⇒「東陽町駅」同名地名のパターン。所在地より東京都と判定。
- 「中津」⇒「中津市」同名の地名が大阪にも存在。これは滋賀県のものと推定。